In [1]:
%matplotlib notebook
from pyfeti.src.utils import DomainCreator, dict2dfmap, create_selection_operator, DofManager
from pyfeti.src.feti_solver import SerialFETIsolver, SolverManager
from pyfeti.cases.case_generator import FETIcase_builder
from scipy import sparse
import amfe
import numpy as np
import matplotlib.pyplot as plt
import copy

def update_system(system_list,V):
    v_dict = manager.vector2localdict(V,manager.global2local_primal_dofs)
    for i,sysi in enumerate(system_list):
        vi = v_dict[i+1]
        sysi.u_output = list(vi.T)
    return system_list

Could not load paradiso!
Could not load paradiso!
Warning! Using python in global path


In [2]:
width = 8.
heigh = 2.
divX=21
divY=6
dobj = DomainCreator(width=width,heigh=heigh,x_divisions=divX,y_divisions=divY)
mesh_file = 'mesh.msh'
dobj.save_gmsh_file(mesh_file)

m1 = amfe.Mesh()
m1.import_msh(mesh_file)


domains_X = 2
domains_Y = 1
base = np.array([1,1,1])
scale_list = 0.5*np.ones(domains_X*domains_Y) + 0.5*np.random.rand(domains_X*domains_Y)
mesh_list = []
facecolor_list = []
scale = 1
count=0
for my in range(domains_Y):
    for mx in range(domains_X):
        mij = m1.translation(np.array([mx*width,my*heigh]))
        mesh_list.append(mij) 
        facecolor_list.append(scale_list[count]*base)
        count+=1
#mesh_list = [m1,m2]

fig1, ax1 = plt.subplots(1,1,figsize=(10,3))

for mi in mesh_list :
    amfe.plot2Dmesh(mi,ax=ax1)

ax1.set_xlim([-2,domains_X*width+2])
ax1.set_ylim([-2,domains_Y*heigh+2])
ax1.set_aspect('equal')
ax1.set_xlabel('Width [m]')
ax1.set_ylabel('Heigh [m]')
plt.legend('off')

<IPython.core.display.Javascript object>

# Solving Static Problem

$$
\begin{bmatrix}
\mathbf{K} & \mathbf{B^T}  \\
\mathbf{B} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u}  \\
\mathbf{\lambda}   
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{f}  \\
\mathbf{0}   
\end{bmatrix}
$$

In [3]:
# creating material
my_material = amfe.KirchhoffMaterial(E=210.0E9, nu=0.3, rho=7.86E3, plane_stress=True, thickness=1.0)

my_system1 = amfe.MechanicalSystem()
my_system1.set_mesh_obj(m1)
my_system1.set_domain(3,my_material)

K1, _ = my_system1.assembly_class.assemble_k_and_f()
M1 = my_system1.assembly_class.assemble_m()


system_list = []
for mi in mesh_list:
    sysi = amfe.MechanicalSystem()
    sysi.set_mesh_obj(mi)
    sysi.set_domain(3,my_material)
    system_list.append(sysi)
#K2, _ = my_system2.assembly_class.assemble_k_and_f()
#M2 = my_system2.assembly_class.assemble_m()

Preallocating the stiffness matrix
Done preallocating stiffness matrix with 100 elements and 252 dofs.
Time taken for preallocation: 0.00 seconds.
Preallocating the stiffness matrix
Done preallocating stiffness matrix with 100 elements and 252 dofs.
Time taken for preallocation: 0.00 seconds.
Preallocating the stiffness matrix
Done preallocating stiffness matrix with 100 elements and 252 dofs.
Time taken for preallocation: 0.00 seconds.


In [4]:
try:
    connectivity = []
    for _,item in m1.el_df.iloc[:, m1.node_idx:].iterrows():
        connectivity.append(list(item.dropna().astype(dtype='int64')))
    m1.el_df['connectivity'] = connectivity
except:
    pass
    
id_matrix = my_system1.assembly_class.id_matrix
id_map_df = dict2dfmap(id_matrix)
s = create_selection_operator(id_map_df,m1.el_df)

neighbors_dict = {}
neighbors_dict['right'] = 2
neighbors_dict['left'] = 1
neighbors_dict['top'] = 5
neighbors_dict['bottom'] = 4
neighbors_dict['bottom_left_corner'] = 6
neighbors_dict['bottom_right_corner'] = 7
neighbors_dict['top_left_corner'] = 8
neighbors_dict['top_right_corner'] = 9

B_local_dict = {}
for key, value in neighbors_dict.items():
    B_local_dict[key] = s.build_B(value)

In [5]:
zeros = np.zeros(K1.shape[0])
case = FETIcase_builder(domains_X,domains_Y, K1, zeros, B_local_dict,s,BC_type='G',force_scaling=1.0)
K_dict, B_dict, f_dict = case.build_subdomain_matrices()

In [6]:
feti_obj = SerialFETIsolver(K_dict,B_dict,f_dict,tolerance=1.0e-6,pseudoinverse_kargs={'method':'splusps','tolerance':1.0E-8})
solution_obj = feti_obj.solve()
u_dict = solution_obj.u_dict

for i,sysi in enumerate(system_list):
    sysi.u_output = [u_dict[i+1]]


In [7]:
p0=1
fig, ax2 = plt.subplots(1,1,figsize=(10,3))
for i,sysi in enumerate(system_list):
    amfe.plot_2D_system_solution(sysi,u_id=0,ax=ax2,factor=p0)

delta_ = 1.5
ax2.set_xlim([-delta_,(2.0+delta_)*width])
ax2.set_ylim([-delta_*3,(1.0+delta_)*heigh ])
ax2.set_aspect('equal')
ax2.set_xlabel('Width [m]')
ax2.set_ylabel('Heigh [m]')
plt.title('Static Problem')
plt.legend('off')

<IPython.core.display.Javascript object>

# Solving Hybrid Eigen-problem

$$
\begin{bmatrix}
\mathbf{K} & \mathbf{B^T}  \\
\mathbf{B} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u}  \\
\mathbf{\lambda}   
\end{bmatrix}
=
\omega^2
\begin{bmatrix}
\mathbf{M} & \mathbf{0}  \\
\mathbf{0} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u}  \\
\mathbf{\lambda}   
\end{bmatrix}
$$

In [8]:
M_dict = {}
for i in range(len(mesh_list)):
    M_dict[i+1] = M1
    
#K_dict = {1:K1, 2:K2}
feti_obj1 = SerialFETIsolver(K_dict,B_dict,f_dict,tolerance=1.0e-12,pseudoinverse_kargs={'method':'splusps','tolerance':1.0E-8})
feti_obj2 = SerialFETIsolver(M_dict,B_dict,f_dict,tolerance=1.0e-12,pseudoinverse_kargs={'method':'splusps','tolerance':1.0E-8})
manager = feti_obj1.manager 
managerM = feti_obj2.manager
Mi = managerM.local_problem_dict[1].K_local
Mi.key_dict = s.selection_dict
Mi.eliminate_by_identity(1,multiplier=0.0)
for i in range(len(mesh_list)):
    M_dict[i+1] = copy.deepcopy(Mi)

In [9]:
B = manager.assemble_global_B()
R = manager.assemble_global_kernel().A
M_,_ = managerM.assemble_global_K_and_f()
K, _ = manager.assemble_global_K_and_f()
M = M_

B_ = B #np.vstack([B,R.T])
BBT_inv = np.linalg.pinv(B_.A.dot(B_.A.T))
scaling = manager.assemble_global_scaling()
S = sparse.diags(1./scaling)
#BBT_inv_tilde = B.dot(S).dot(S.dot(B.T))
BBT_inv_tilde = BBT_inv
P = np.eye(M.shape[0]) - B_.A.T.dot(BBT_inv_tilde.dot(B_.A))
u0 = solution_obj.displacement 
nmodes = 30

In [10]:
P = sparse.csr_matrix(P)
P

<504x504 sparse matrix of type '<class 'numpy.float64'>'
	with 528 stored elements in Compressed Sparse Row format>

In [11]:
Pa = sparse.eye(M.shape[0]) - S.dot(B_.T.dot(B_))


In [12]:
error = Pa - P
print(error.max())
print(error.min())

0.0
0.0


In [13]:
P

<504x504 sparse matrix of type '<class 'numpy.float64'>'
	with 528 stored elements in Compressed Sparse Row format>

# Solving Hybrid Eigen-problem using the Primal Assembly Operator

$$
\begin{bmatrix}
\mathbf{L}^T \mathbf{K} \mathbf{L}^T 
\end{bmatrix}
\begin{bmatrix}
\mathbf{u}  \\
\end{bmatrix}
=
\omega^2
\begin{bmatrix}
\mathbf{L}^T \mathbf{M} \mathbf{L} 
\end{bmatrix}
\begin{bmatrix}
\mathbf{u}  \\
\end{bmatrix}
$$

In [14]:
L = manager.assemble_global_L()
Lexp = manager.assemble_global_L_exp()
Kp = L.dot(K.dot(Lexp))
Mp = L.dot(M.dot(Lexp))
#Dp = np.linalg.inv(Kp).dot(Mp)
lu_p = sparse.linalg.splu(Kp)
Dp = sparse.linalg.LinearOperator(shape=Kp.shape,matvec = lambda x : lu_p.solve((Mp.dot(x))))

/home/ge72tih/miniconda3/lib/python3.7/site-packages/scipy-1.2.1-py3.7-linux-x86_64.egg/scipy/sparse/linalg/dsolve/linsolve.py:296: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [15]:
eigval_, Vp = sparse.linalg.eigs(Dp,k=nmodes,v0=L.dot(u0))
eigval_

val_p = np.sort(1/eigval_)
val_p

freq_p = np.sqrt(val_p)/(2.0*np.pi)
freq_p

array([   6.52118721+0.j,   38.33199758+0.j,   80.90701055+0.j,
         98.70339441+0.j,  175.47922268+0.j,  242.58780431+0.j,
        262.9452458 +0.j,  357.16953044+0.j,  403.7882929 +0.j,
        455.83580267+0.j,  557.32137757+0.j,  563.86792963+0.j,
        660.14509228+0.j,  721.47441055+0.j,  761.18134302+0.j,
        833.22246052+0.j,  852.30104793+0.j,  873.57571666+0.j,
        902.87033627+0.j,  939.56618785+0.j, 1005.12876201+0.j,
       1013.40007243+0.j, 1039.01743902+0.j, 1117.70068496+0.j,
       1129.04850694+0.j, 1148.28558395+0.j, 1185.89293547+0.j,
       1219.01920121+0.j, 1235.81607622+0.j, 1263.73326255+0.j])

In [16]:
plt.figure()
plt.bar(list(range(len(freq_p))),freq_p)
plt.title('Frequency [Hz]')

<IPython.core.display.Javascript object>

/home/ge72tih/miniconda3/lib/python3.7/site-packages/matplotlib-3.0.3-py3.7-linux-x86_64.egg/matplotlib/transforms.py:827: ComplexWarning: Casting complex values to real discards the imaginary part
  points = np.array(args, dtype=float).reshape(2, 2)


Text(0.5, 1.0, 'Frequency [Hz]')

In [17]:
Vprimal = Lexp.dot(Vp)

In [18]:
facecolor_list

[array([0.70501791, 0.70501791, 0.70501791]),
 array([0.65638963, 0.65638963, 0.65638963])]

In [19]:
sysi = update_system(system_list,Vprimal)

In [20]:
fig, ax3_list = plt.subplots(3,3,figsize=(10,10))
counter = 0
delta_ = 1.0
for ax_ij in ax3_list:
    for ax2 in ax_ij:
        for i,sysi in enumerate(system_list):
            amfe.plot_2D_system_solution(sysi,u_id=(nmodes - 1 - counter),ax=ax2,factor=20,
                        facecolor=tuple(facecolor_list[i]),linewidth=0.1)
    
        delta = 3
        ax2.set_xlim([-delta,domains_X*width+delta])
        ax2.set_ylim([-delta,domains_Y*heigh+delta])
        
        ax2.set_aspect('equal')
        ax2.set_xlabel('Width [m]')
        ax2.set_ylabel('Heigh [m]')
        ax2.set_title('Mode id = %i' %(counter+1) )
        counter+=1
plt.legend('off')
#plt.tight_layout()
plt.subplots_adjust(hspace=-0.70, wspace=0.35)

<IPython.core.display.Javascript object>

# Solving Hybrid Eigen-problem using the inverse iteration =

$$
\begin{bmatrix}
\mathbf{K} & \mathbf{B^T}  \\
\mathbf{B} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u_{n+1}}  \\
\mathbf{\lambda_{n+1}}   
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{M} & \mathbf{0}  \\
\mathbf{0} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u_{n}}  \\
\mathbf{\lambda_{n}}   
\end{bmatrix}
$$

using initial random guess

In [21]:
countswp=0
def system_without_projection(u,tol=1.0e-8):
    
    global countswp
    f = M.dot(u)
    f_dict = manager.vector2localdict(f,manager.global2local_primal_dofs)
    feti_obj = SerialFETIsolver(K_dict,B_dict,f_dict,tolerance=tol,pseudoinverse_kargs={'method':'splusps','tolerance':1.0E-8})
    solution_obj = feti_obj.solve()
    u_dict = solution_obj.u_dict
    countswp+=1
    return solution_obj.displacement
    
#un = system_without_projection(u0)
D_wp = sparse.linalg.LinearOperator(shape=M.shape,matvec = lambda x : system_without_projection(x))

In [22]:
np.random.seed(1)
u0 = np.random.rand(D_wp.shape[0])
u0 /= np.linalg.norm(u0)

In [23]:
np.linalg.norm(u0)

1.0

In [24]:
eigval_without_projection_, V_wp_ = sparse.linalg.eigs(D_wp,k=nmodes,v0=u0,maxiter=200)
val_wp_ = np.sort(1/eigval_without_projection_.real)
freq_dual_wp_ = np.sqrt(val_wp_)/(2.0*np.pi)
freq_dual_wp_

array([   6.52111964,   38.3207462 ,   80.94436818,   98.68500742,
        175.33911223,  243.39495043,  262.63396935,  358.31105546,
        401.38827863,  455.36354967,  553.44376112,  559.09430211,
        655.72259214,  717.01237155,  765.38142159,  820.29025049,
        852.26584623,  867.11750953,  898.03340711,  934.3695231 ,
        993.90381219,  993.90381219, 1043.38532625, 1111.67800599,
       1136.15366699, 1136.15366699, 1194.67688401, 1194.67688401,
       1194.84984282, 1232.9633983 ])

In [25]:
countswp

106

In [26]:
plt.figure()
plt.bar(list(range(len(freq_dual_wp_))),freq_dual_wp_)
plt.title('Frequency [Hz]')

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Frequency [Hz]')

In [27]:
plt.figure()
plt.plot(freq_p,freq_dual_wp_,'o')
plt.xlabel('Frequency Primal')
plt.ylabel('Frequency Dual without projection with initial random guess')

<IPython.core.display.Javascript object>

/home/ge72tih/miniconda3/lib/python3.7/site-packages/numpy-1.16.2-py3.7-linux-x86_64.egg/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


Text(0, 0.5, 'Frequency Dual without projection with initial random guess')

In [28]:
shift=0
#error = 100*np.abs(freq_p[:-shift] - freq_dual_wp_[shift:])/freq_p[:-shift]
error = 100*np.abs(freq_p - freq_dual_wp_)/freq_p
max_error = error.max()
print(max_error)

(3.314913455282747+0j)


In [29]:
np.mean(error )

(0.7527850436930505+0j)

In [30]:
plt.figure()
#plt.plot(error,'o')
plt.bar(list(range(1,len(error)+1)),error)
plt.xlabel('Mode index')
plt.ylabel('Eigenfrequency error (%)')
plt.title('Error between Primal vs Classical FETI')


<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Error between Primal vs Classical FETI')

In [31]:
sysi = update_system(system_list,V_wp_)


In [32]:
fig, ax1_list = plt.subplots(3,3,figsize=(10,10))
counter = 0
delta_ = 1.0
for ax_ij in ax1_list:
    for ax2 in ax_ij:
        for i,sysi in enumerate(system_list):
            amfe.plot_2D_system_solution(sysi,u_id=(nmodes -1 - counter),ax=ax2,factor=20,
             facecolor=tuple(facecolor_list[i]),linewidth=0.1)
    
        delta = 3
        ax2.set_xlim([-delta,domains_X*width+delta])
        ax2.set_ylim([-delta,domains_Y*heigh+delta])
        
        ax2.set_aspect('equal')
        ax2.set_xlabel('Width [m]')
        ax2.set_ylabel('Heigh [m]')
        ax2.set_title('Mode id = %i' %(counter+1) )
        counter+=1
plt.legend('off')
#plt.tight_layout()
plt.subplots_adjust(hspace=-0.70, wspace=0.35)

<IPython.core.display.Javascript object>

# Solving Hybrid Eigen-problem using the inverse iteration =

$$
\begin{bmatrix}
\mathbf{K} & \mathbf{B^T}  \\
\mathbf{B} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u_{n+1}}  \\
\mathbf{\lambda_{n+1}}   
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{M} & \mathbf{0}  \\
\mathbf{0} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u_{n}}  \\
\mathbf{\lambda_{n}}   
\end{bmatrix}
$$

using initial guess $v_0 = P v$

In [33]:
eigval_without_projection, V_wp = sparse.linalg.eigs(D_wp,k=nmodes,v0 = P.dot(u0))
val_wp = np.sort(1/eigval_without_projection.real)
freq_dual_wp = np.sqrt(val_wp)/(2.0*np.pi)
freq_dual_wp

array([   6.5211193 ,   38.32093214,   80.94315627,   98.68524959,
        175.34087755,  243.39961966,  262.64209086,  358.27458804,
        401.41225098,  455.38307927,  553.43154847,  559.11942697,
        655.76928905,  717.06738553,  765.34540845,  820.25376953,
        852.28319568,  867.0829116 ,  898.01755678,  934.30569408,
        993.95347841,  993.95347841, 1043.45662069, 1112.74274669,
       1136.1753622 , 1136.1753622 , 1188.54207522, 1196.75866528,
       1196.75866528, 1232.86362438])

In [34]:
plt.figure()
plt.bar(list(range(len(freq_dual_wp))),freq_dual_wp)
plt.title('Frequency [Hz]')

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Frequency [Hz]')

In [35]:
plt.figure()
plt.plot(freq_p,freq_dual_wp,'o')
plt.xlabel('Frequency Primal')
plt.ylabel('Frequency Dual without projection')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Frequency Dual without projection')

In [36]:
error = 100*np.abs(freq_p - freq_dual_wp)/freq_p
max_error = error.max()
print(max_error)

(3.1604549969060005+0j)


In [37]:
np.mean(error )

(0.7206340158732001+0j)

In [38]:
plt.figure()
plt.plot(error,'o')

<IPython.core.display.Javascript object>

In [39]:
sysi = update_system(system_list,V_wp)

In [40]:
fig, ax1_list = plt.subplots(3,3,figsize=(10,10))
counter = 0
delta_ = 1.0
for ax_ij in ax1_list:
    for ax2 in ax_ij:
        for i,sysi in enumerate(system_list):
            amfe.plot_2D_system_solution(sysi,u_id=(nmodes - 1 - counter),ax=ax2,factor=10)
        #ax2.set_xlim([-delta_*width,(2.0+delta_)*width])
        #ax2.set_ylim([-delta_*heigh,(1.0+delta_)*heigh ])
        ax2.set_aspect('equal')
        ax2.set_xlabel('Width [m]')
        ax2.set_ylabel('Heigh [m]')
        ax2.set_title('Mode id = %i' %(counter+1) )
        counter+=1
plt.legend('off')
plt.tight_layout()

<IPython.core.display.Javascript object>

# Solving Hybrid Eigen-problem using the inverse iteration =

$$
\begin{bmatrix}
\mathbf{K} & \mathbf{B^T}  \\
\mathbf{B} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u_{n+1}}  \\
\mathbf{\lambda_{n+1}}   
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{P}^T \mathbf{M}\mathbf{P} & \mathbf{0}  \\
\mathbf{0} & \mathbf{0}  
\end{bmatrix}
\begin{bmatrix}
\mathbf{u_{n}}  \\
\mathbf{\lambda_{n}}   
\end{bmatrix}
$$

using initial guess $v_0 = P v$

In [41]:
#u0 = solution_obj.displacement 
counts=0
def system(u,tol=1.0e-8):
    global counts
    f = P.T.dot(M.dot(P.dot(u)))
    #f = M.dot(u)
    f_dict = manager.vector2localdict(f,manager.global2local_primal_dofs)
    feti_obj = SerialFETIsolver(K_dict,B_dict,f_dict,tolerance=tol,pseudoinverse_kargs={'method':'splusps','tolerance':1.0E-8})
    solution_obj = feti_obj.solve()
    u_dict = solution_obj.u_dict
    counts+=1
    return solution_obj.displacement
    




In [42]:
D = sparse.linalg.LinearOperator(shape=M.shape,matvec = lambda x : system(x))
eigval, V = sparse.linalg.eigs(D,k=nmodes,v0 = P.dot(u0),maxiter=200)
#eigval, V = sparse.linalg.eigs(D,k=nmodes,v0 = u0,maxiter=200)
val = np.sort(1/eigval.real)
freq_dual = np.sqrt(val)/(2.0*np.pi)
freq_dual

array([   6.52111947,   38.32090099,   80.9430616 ,   98.68518416,
        175.33700498,  243.36199222,  262.62991746,  358.18496569,
        401.58156496,  455.57433633,  553.78067185,  559.19001702,
        656.1160543 ,  717.83658177,  764.9126062 ,  821.58514093,
        852.38969009,  868.63430063,  898.67549123,  934.90359915,
        994.30287877, 1000.06734922, 1043.24352282, 1118.05671917,
       1133.41332928, 1137.84937733, 1198.25281833, 1198.25281833,
       1237.24905324, 1237.24905324])

In [43]:
counts

109

In [44]:
up = P.dot(u0)

In [45]:
P

<504x504 sparse matrix of type '<class 'numpy.float64'>'
	with 528 stored elements in Compressed Sparse Row format>

In [46]:
B.dot(up)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [47]:
B.dot(V[:,0])

array([-1.42648995e-06+0.j, -5.43304183e-07+0.j, -1.82678838e-06+0.j,
       -1.87739789e-07+0.j,  5.24979029e-06+0.j,  5.23497794e-07+0.j,
       -2.17507916e-06+0.j,  5.70399914e-07+0.j,  3.04039074e-06+0.j,
       -4.57632494e-08+0.j, -2.86182354e-06+0.j, -3.17090487e-07+0.j])

In [48]:
plt.figure()
plt.bar(list(range(len(freq_dual))),freq_dual)
plt.title('Frequency [Hz]')

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Frequency [Hz]')

In [49]:
plt.figure()
plt.plot(freq_p,freq_dual,'o')
plt.xlabel('Frequency Primal')
plt.ylabel('Frequency Dual with Projection')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Frequency Dual with Projection')

In [50]:
error = 100*np.abs(freq_p - freq_dual)/freq_p
max_error = error.max()
print(max_error)

(2.095711974110758+0j)


In [51]:
np.mean(error )

(0.5529563584703932+0j)

In [52]:
plt.figure()
#plt.plot(error,'o')
plt.bar(list(range(1,len(error)+1)),error)
plt.xlabel('Mode index')
plt.ylabel('Eigenfrequency error (%)')
plt.title('Error between Primal vs Projected FETI')

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Error between Primal vs Projected FETI')

In [53]:
sysi = update_system(system_list,V.real)

In [54]:
fig, ax3_list = plt.subplots(3,3,figsize=(10,10))
counter = 0
delta_ = 1.0
for ax_ij in ax3_list:
    for ax2 in ax_ij:
        for i,sysi in enumerate(system_list):
            #amfe.plot_2D_system_solution(sysi,u_id=(nmodes - 1 - counter),ax=ax2,factor=10,
            #                             facecolor=tuple(facecolor_list[i]),linewidth=0.1)
            amfe.plot_2D_system_solution(sysi,u_id=(nmodes - 1 - counter),ax=ax2,factor=10)
        #ax2.set_xlim([-delta_*width,(2.0+delta_)*width])
        #ax2.set_ylim([-delta_*heigh,(1.0+delta_)*heigh ])
        delta = 3
        ax2.set_xlim([-delta,domains_X*width+delta])
        ax2.set_ylim([-delta,domains_Y*heigh+delta])
        #ax2.set_ylim([-2,domains_X*width+2])
        
        ax2.set_aspect('equal')
        ax2.set_xlabel('Width [m]')
        ax2.set_ylabel('Heigh [m]')
        ax2.set_title('Mode index = %i' %(counter+1) )
        counter+=1
plt.legend('off')
#plt.tight_layout()
plt.subplots_adjust(hspace=-0.70, wspace=0.35)

<IPython.core.display.Javascript object>

In [55]:
#fig.savefig('projected_dual_modes_v1.png',dpi=1000)

In [56]:
plt.figure()
plt.imshow(B.A)

<IPython.core.display.Javascript object>

In [57]:
plt.figure()
plt.imshow(B.dot(B.T))

<IPython.core.display.Javascript object>

TypeError: Image data cannot be converted to float

In [ ]:
np.diag(B.dot(B.T))

In [ ]:
l = [np.array([1,1]),np.array([1,1])]

In [ ]:
np.concatenate(l)

In [ ]:
scaling = manager.assemble_global_scaling()

In [ ]:
plt.figure(0)
plt.plot(scaling,'o')

In [ ]:
plt.figure()
plt.imshow(BBT_inv)

In [ ]:
BBT_inv

In [ ]:
S = np.diag(1./scaling)
S*B.T.dot(B)

In [ ]:
plt.figure()
plt.imshow(B.dot(S.dot(B.T)))

In [ ]:
BB_inv_tilde = B.dot(S).dot(S.dot(B.T))

In [ ]:
mode_id = 3
plt.figure()
plt.plot(np.sort(np.abs(V[:,nmodes-mode_id-1].real)),np.sort(np.abs(Vprimal[:,nmodes-mode_id-1])),'o')

In [ ]:
#mode_id = 0
plt.figure()
plt.plot(np.sort(np.abs(V_wp_[:,nmodes-mode_id-1].real)),np.sort(np.abs(Vprimal[:,nmodes-mode_id-1])),'o')

In [ ]:
error = BB_inv_tilde - BBT_inv

In [ ]:
error.max()

In [ ]:
error.min()

In [ ]:
V_wp_.real

In [ ]:
V.real

In [ ]:
freq_

In [ ]:

mode_id = 1
plt.figure()
plt.plot(V_wp_.real[:,mode_id],V.real[:,mode_id],'o')

In [ ]:
up = P.dot(u0)

In [ ]:
B.dot(up)

In [ ]:
up2 = P.dot(up)

In [ ]:
up2

In [ ]:
up2-up